In [ ]:
!pip install pytorch_lightning
!pip install torchsummaryX
!pip install webdataset
#!pip install datasets
#!git clone https://github.com/black0017/MedicalZooPytorch.git
#!pip install -r MedicalZooPytorch/installation/requirements.txt
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 675kB 7.2MB/s 
     |████████████████████████████████| 276kB 42.0MB/s 
     |████████████████████████████████| 829kB 37.5MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=94792cdd571692b637c07a6f6f0404a2f30aaffe8a475e4fc6775bfff7c5420d
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d3773dfe9214bf628d9dbcd738fde8b03a964ac789b203ca8c52ea02c8e5e387
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully unin

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/MacAI

/content/drive/MyDrive/MacAI


In [ ]:
import os
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
import sys

#from lib import medzoo
import nibabel as nb
from skimage import transform
import matplotlib.pyplot as plt
from lib.losses3D.basic import compute_per_channel_dice, expand_as_one_hot
from lib.medzoo.Unet3D import UNet3D

In [ ]:
from io import BytesIO
from nibabel import FileHolder, Nifti1Image
import os
import torch
from skimage import transform
import webdataset as wds

In [ ]:
def col_img(batch):
    bytes_data_list = [list(batch[i].items())[1][1] for i in range(5)]
    bb = BytesIO(bytes_data_list[0])
    fh = FileHolder(fileobj=bb)

    f_flair = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_list[1])
    fh = FileHolder(fileobj=bb)
    f_seg = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_list[2])
    fh = FileHolder(fileobj=bb)
    f_t1 = Nifti1Image.from_file_map({'header': fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_list[3])
    fh = FileHolder(fileobj=bb)
    f_t1ce=Nifti1Image.from_file_map({'header':fh, 'image':fh}).get_fdata()
    bb = BytesIO(bytes_data_list[4])
    fh = FileHolder(fileobj=bb)
    f_t2 =Nifti1Image.from_file_map({'header':fh, 'image':fh}).get_fdata()

    f_flair = torch.as_tensor(np.expand_dims(np.pad(f_flair, [(0, 0), (0, 0), (2, 3)]), axis=0)/1000).half()
    f_t1 = torch.as_tensor(np.expand_dims(np.pad(f_t1, [(0, 0), (0, 0), (2, 3)]), axis=0)/1000).half()
    f_t2 = torch.as_tensor(np.expand_dims(np.pad(f_t2, [(0, 0), (0, 0), (2, 3)]), axis=0)/1000).half()
    f_t1ce = torch.as_tensor(np.expand_dims(np.pad(f_t1ce, [(0, 0), (0, 0), (2, 3)]), axis=0)/1000).half()

    orig_f_seg = np.expand_dims(np.pad(f_seg, [(0, 0), (0, 0), (2, 3)]), axis=0)
    concat = torch.cat([f_t1, f_t1ce, f_t2, f_flair], axis=0)
    f_seg = torch.as_tensor(orig_f_seg) #.half()

    # remove nan?
    f_seg[f_seg != f_seg] = 0
    concat[concat != concat] = 0

    return ([concat, f_seg])
  


In [ ]:
def selector_train(x):
  folder_name = list(x.items())[0][1].split('/')[-1]

  if len(folder_name) >= 17:
    return int(folder_name[17]) < 3
  else:
    return False
  
def selector_eval(x):
  folder_name = list(x.items())[0][1].split('/')[-1]
  if len(folder_name) >= 17:
    return int(folder_name[17]) == 3
  else:
    return False
  

In [ ]:
train_dataset = wds.Dataset("macai_datasets/brats_training.tar.gz")
eval_dataset = wds.Dataset("macai_datasets/brats_training.tar.gz")

#Very janky way of seperating into train and validation 
eval_dataset.select(selector_eval)
train_dataset.select(selector_train)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5,collate_fn=col_img)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=5,collate_fn=col_img)

In [ ]:
#Pytorch-lightning setup
class TumourSegmentation(pl.LightningModule):
  def __init__(self,model):
    super().__init__()
    self.model = model
  
  def forward(self,x):
  #  x=x.half()

    f = self.model.forward(x)

  #  print('Done forward step!')
    return f

  def training_step(self, batch, batch_idx):
    x, y = batch
    x = torch.unsqueeze(x, axis=0)
    y = torch.unsqueeze(y, axis=0)
    #print(x.shape)

    y_hat = self.forward(x)

    #plt.imshow(y_hat[0, 0, 120], cmap='')
    #plt.imshow(y_hat.cpu()[0, 1, 120])
    #plt.imshow(y_hat[0, 2, 120])
    #plt.imshow(y_hat[0, 3, 120])
    #plt.show()

    shape = list(y.size())
    shape[1] = 3
    zeros = torch.zeros(shape).cuda()

    for i in range(1, 4):
      zeros[:, i-1][torch.squeeze(y == i, dim=1)] = 1

  # basic mean of all channels for now
    loss = compute_per_channel_dice(y_hat, zeros)
    loss[loss != loss] = 0
    print('Training loss: ')
    print(loss)
    loss = torch.sum(loss)
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    x = torch.unsqueeze(x, axis=0)
    y = torch.unsqueeze(y, axis=0)
    #print(x.shape)

    y_hat = self.forward(x)

    #plt.imshow(y_hat[0, 0, 120], cmap='')
    #plt.imshow(y_hat.cpu()[0, 1, 120])
    #plt.imshow(y_hat[0, 2, 120])
    #plt.imshow(y_hat[0, 3, 120])
    #plt.show()

    shape = list(y.size())
    shape[1] = 3
    zeros = torch.zeros(shape).cuda()

    for i in range(1, 4):
      zeros[:, i-1][torch.squeeze(y == i, dim=1)] = 1

  # basic mean of all channels for now
    loss = compute_per_channel_dice(y_hat, zeros)
    loss[loss != loss] = 0
    print('Validation loss: ')
    print(loss)
    loss = torch.sum(loss)
    self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    return loss

  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=0.02)


In [ ]:
#model initialization

unet_model =  UNet3D(in_channels=4, n_classes=3, base_n_filter=8) #.cuda()
model = TumourSegmentation(unet_model)

In [ ]:
#Training
trainer = pl.Trainer(
    accumulate_grad_batches = 1,
    gpus = 1,
    max_epochs = 1,
    precision=16,
    check_val_every_n_epoch = 1,

)
#, eval_dataloader
#trainer.tune()
trainer.fit(model, train_dataloader, eval_dataloader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name  | Type   | Params
---------------------------------
0 | model | UNet3D | 1.8 M 
---------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


tensor([0.0296, 0.0015, 0.0000], device='cuda:0')
tensor([0.0188, 0.0094, 0.0000], device='cuda:0')


tensor([ 0.0098, -0.0021,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0155, -0.0198,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0007, -0.0027,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0111, -0.0300,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0134, -0.0013,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0288, -0.0210,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0069, -0.0127,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0003, -0.0074,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0293, -0.0214,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0010, -0.0068,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([-0.0002, -0.0016,  0.0000], device='cuda:0',
       grad_fn=<IndexPutBackward>)
tensor([ 0.0015, -0.0011,  0.000

IndexError: ignored

In [ ]:
    # to be used later
#def bbox2_3D(img):

#    r = np.any(img, axis=(1, 2))
#    c = np.any(img, axis=(0, 2))
#    z = np.any(img, axis=(0, 1))

#    rmin, rmax = np.where(r)[0][[0, -1]]
#    cmin, cmax = np.where(c)[0][[0, -1]]
#    zmin, zmax = np.where(z)[0][[0, -1]]

#    return rmin, rmax, cmin, cmax, zmin, zmax
    



      xmins = []
      xmaxs = []
      ymins = []
      ymaxs = []
      zmins = []
      zmaxs = []
      for img in [f_flair, f_seg, f_t1ce, f_t1, f_t2]:
        xmin, xmax, ymin, ymax, zmin, zmax = bbox2_3D(img)
        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)
        zmins.append(zmin)
        zmaxs.append(zmax)
      xmin = np.min(xmin)
      ymin = np.min(ymin)
      zmin = np.min(zmin)
      xmax = np.max(xmax)
      ymax = np.max(ymax)
      zmax = np.max(zmax)

      f_flair = f_flair[xmin:xmax, ymin:ymax, zmin:zmax]
      f_seg = f_seg[xmin:xmax, ymin:ymax, zmin:zmax]
      f_t1ce = f_t1ce[xmin:xmax, ymin:ymax, zmin:zmax]
      f_t1 = f_t1[xmin:xmax, ymin:ymax, zmin:zmax]
      f_t2 = f_t2[xmin:xmax, ymin:ymax, zmin:zmax]

      print(f_flair.shape)